<table align="left"><td><a target="_blank" href="https://colab.research.google.com/github/superannotateai/model-deployment-tutorials/blob/main/OAK/SuperAnnotate_OAK_Deeplabv3%2B_Deployment.ipynb"><img src="https://user-images.githubusercontent.com/25985824/104791629-6e618700-5769-11eb-857f-d176b37d2496.png" height="32" width="32"> Try in Google Colab</a></td></table>

# 1. Install Prerequisites 
(Please click on **RESTART RUNTIME** button when it appears in the output of this code block)



In [ ]:
!sudo apt-get install -y pciutils cpio
!sudo apt autoremove
!wget http://registrationcenter-download.intel.com/akdlm/irc_nas/16345/l_openvino_toolkit_p_2020.1.023.tgz
path = "l_openvino_toolkit_p_2020.1.023.tgz"
!tar xf "{path}"
%cd l_openvino_toolkit_p_2020.1.023/
!./install_openvino_dependencies.sh && \
    sed -i 's/decline/accept/g' silent.cfg && \
    ./install.sh --silent silent.cfg
!bash /content/l_openvino_toolkit_p_2020.1.023/install_openvino_dependencies.sh
!bash /opt/intel/openvino/deployment_tools/model_optimizer/install_prerequisites/install_prerequisites.sh

%cd /content/

!pip install tensorflow-gpu==1.15.3
!pip install tf_slim==1.0.0

!git clone https://github.com/nolanliou/mobile-deeplab-v3-plus
!pip install superannotate
!pip install google-resumable-media==0.5.0

# 2. Setup your SuperAnnotate token, project names of trianing images and desired output model name

In [ ]:
TOKEN = "For Pro Users, the Token-ID can be generated in the teams section"
PROJECT_NAME = "City"
OUTPUT_MODEL = "sa-tutorial"

# 3. Download and preprocess the data for training

In [ ]:
import superannotate as sa
import json
import os 
from shutil import copy, make_archive
import re

proj_folder = dict(zip(PROJECT_NAME, ["proj_" + str(i) for i in range(len(PROJECT_NAME))]))
token_json = {"token": TOKEN}
with open('sa_config.json', 'w') as f:
  json.dump(token_json, f)

sa.init('sa_config.json')

dataset_base_dir = '/content/mobile-deeplab-v3-plus/datasets/pascal_voc_seg'
if not os.path.exists(dataset_base_dir):
  os.mkdir(dataset_base_dir)

export = sa.prepare_export(PROJECT_NAME, annotation_statuses=["Completed"], include_fuse=True)
sa.download_export(PROJECT_NAME, export, dataset_base_dir)

if not os.path.exists(dataset_base_dir + '/VOC2012'):
  os.makedirs(dataset_base_dir + '/VOC2012/JPEGImages')
  os.makedirs(dataset_base_dir + '/VOC2012/SegmentationClass')
  os.makedirs(dataset_base_dir + '/VOC2012/ImageSets/Segmentation')
  os.makedirs(dataset_base_dir + '/tfrecord')

%cd /content/mobile-deeplab-v3-plus/datasets/pascal_voc_seg/
!rm -f *.json
!rm -f *___save.png

folder_content = os.listdir('.')
for entry in folder_content:
  if len(entry.split('.'))==1:
    continue
  if entry.endswith('___fuse.png'):
    im_name, _ = os.path.splitext(entry[:-11])
    copy(entry, 'VOC2012/SegmentationClass/' + im_name + '.png')
  else:
    copy(entry, 'VOC2012/JPEGImages/' + entry)

with open(dataset_base_dir + '/VOC2012/ImageSets/Segmentation/trainval.txt', 'w') as f:
  image_names = os.listdir(dataset_base_dir + '/VOC2012/JPEGImages')
  for image_name in image_names:
    file_name, _ = os.path.splitext(image_name)
    f.write(file_name + '\n')
  
data_size = len(image_names)
val_size = int(data_size * 0.1)
train_size = data_size - val_size
!shuf VOC2012/ImageSets/Segmentation/trainval.txt | tee >(head -n "{val_size}" > VOC2012/ImageSets/Segmentation/val.txt) | tail -n "{train_size}" > VOC2012/ImageSets/Segmentation/train.txt
!python ../remove_gt_colormap.py --original_gt_folder="VOC2012/SegmentationClass" --output_dir="VOC2012/SegmentationClassRaw"

%cd /content/mobile-deeplab-v3-plus/datasets/

with open('build_voc2012_data.py') as f:
  record_gen = f.read()

record_gen = re.sub(r"import pdb\; pdb\.set\_trace\(\)\n  ", "", record_gen)

with open('build_voc2012_data.py', 'w') as f:
  f.write(record_gen)

!python build_voc2012_data.py \
  --image_folder="pascal_voc_seg/VOC2012/JPEGImages" \
  --semantic_segmentation_folder="pascal_voc_seg/VOC2012/SegmentationClassRaw" \
  --list_folder="pascal_voc_seg/VOC2012/ImageSets/Segmentation" \
  --image_format="jpg" \
  --output_dir="pascal_voc_seg/tfrecord/"

%cd /content/mobile-deeplab-v3-plus/
objdata = 'segmentation_dataset.py'
with open(objdata) as f:
    s = f.read()

with open('/content/mobile-deeplab-v3-plus/datasets/pascal_voc_seg/classes/classes.json', 'r') as f:
  class_data = json.load(f)

num_classes = len(class_data) + 1
s = re.sub('num_classes=21','num_classes=' + str(num_classes),s)
s = re.sub("'train': 1464","'train': " + str(train_size),s)
s = re.sub("'trainval': 2913","'trainval': " + str(data_size),s)
s = re.sub("'val': 1449","'val': " + str(val_size),s)

with open(objdata, 'w') as f:
  f.write(s)

# 4. Train the model

In [ ]:
%env PYTHONPATH=/content/mobile-deeplab-v3-plus:/env/python

import tensorflow as tf
from mobilenet_v2 import MobilenetV2
import urllib

%cd /content/mobile-deeplab-v3-plus/datasets/pascal_voc_seg/
if not os.path.exists('/content/mobile-deeplab-v3-plus/datasets/pascal_voc_seg/mnv2'):
  os.mkdir('/content/mobile-deeplab-v3-plus/datasets/pascal_voc_seg/mnv2')

%cd mnv2 
!wget https://storage.googleapis.com/mobilenet_v2/checkpoints/mobilenet_v2_1.0_224.tgz
!tar xvf mobilenet_v2_1.0_224.tgz

if not os.path.exists('/content/mobile-deeplab-v3-plus/datasets/pascal_voc_seg/mnv2_adapted'):
  os.mkdir('/content/mobile-deeplab-v3-plus/datasets/pascal_voc_seg/mnv2_adapted')

%cd /content/mobile-deeplab-v3-plus/

tf.reset_default_graph()

# For simplicity we just decode jpeg inside tensorflow.
# But one can provide any input obviously.
file_input = tf.placeholder(tf.string, ())

image = tf.image.decode_jpeg(tf.read_file(file_input))

images = tf.expand_dims(image, 0)
images = tf.cast(images, tf.float32) / 128. - 1
images.set_shape((None, None, None, 3))
images = tf.image.resize_images(images, (224, 224))
mobilenet_model = MobilenetV2()

logits, endpoints = mobilenet_model.forward(images, is_training=False)

exclude = ['global_step']
variables_to_restore = tf.contrib.slim.get_variables_to_restore(exclude=exclude)
variables_map = {}
for v in variables_to_restore:
  old_name = v.name.split(':')[0]
  old_name = old_name.replace('conv2d/kernel', 'weights')
  old_name = old_name.replace('conv2d/bias', 'biases')
  variables_map[old_name] = v
tf.train.init_from_checkpoint('/content/mobile-deeplab-v3-plus/datasets/pascal_voc_seg/mnv2/mobilenet_v2_1.0_224.ckpt', variables_map)
filename, _ = urllib.request.urlretrieve('https://upload.wikimedia.org/wikipedia/commons/f/fe/Giant_Panda_in_Beijing_Zoo_1.JPG')

saver = tf.train.Saver()
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  x = endpoints['Predictions'].eval(feed_dict={file_input: filename})
  saver.save(sess, '/content/mobile-deeplab-v3-plus/datasets/pascal_voc_seg/mnv2_adapted/mobilenet-v2-224.ckpt')

if not os.path.exists("/content/mobile-deeplab-v3-plus/datasets/pascal_voc_seg/checkpoint"):
  os.mkdir("/content/mobile-deeplab-v3-plus/datasets/pascal_voc_seg/checkpoint")

train_iters = 15000
%cd /content/mobile-deeplab-v3-plus/
!python run.py --dataset_dir="/content/mobile-deeplab-v3-plus/datasets/pascal_voc_seg/tfrecord"\
  --dataset_name="pascal_voc2012"\
  --logdir="/content/mobile-deeplab-v3-plus/datasets/pascal_voc_seg/checkpoint"\
  --model_type="deeplab-v3-plus"\
  --train_subset="train"\
  --base_learning_rate=0.001\
  --num_clones=1\
  --model_input_size=256\
  --model_input_size=256\
  --training_number_of_steps="{train_iters}"\
  --pretrained_backbone_model_dir="/content/mobile-deeplab-v3-plus/datasets/pascal_voc_seg/mnv2_adapted"

# 5. Generate and download the files needed for the OAK-D device

In [ ]:
import requests
from google.colab import files

if not os.path.exists('/content/export'):
  os.mkdir('/content/export')

!python run.py --dataset_dir="/content/mobile-deeplab-v3-plus/datasets/pascal_voc_seg/tfrecord"\
  --dataset_name="pascal_voc2012"\
  --logdir="/content/mobile-deeplab-v3-plus/datasets/pascal_voc_seg/checkpoint"\
  --model_type="deeplab-v3-plus"\
  --model_input_size=256\
  --model_input_size=256\
  --mode=export\
  --export_dir="/content/export"

!python freeze.py --model_dir="/content/export"\
  --output_node_names=Output \
  --output_dir="/content"

%cd /content/
!source /opt/intel/openvino/bin/setupvars.sh && \
    python /opt/intel/openvino/deployment_tools/model_optimizer/mo_tf.py \
--input_model /content/frozen_model.pb \
--input_shape [1,256,256,3] \
--mean_values=[127.5,127.5,127.5] \
--scale_values=[255] \
--reverse_input_channel \
--data_type FP16

os.mkdir("/content/" + OUTPUT_MODEL)
blob_dir = "/content/" + OUTPUT_MODEL

binfile = "/content/frozen_model.bin"
xmlfile = "/content/frozen_model.xml"

url = "http://69.164.214.171:8083/compile"
params = {
    'version': '2020.1'
}
payload = {
    'compile_type': 'myriad', 
    'compiler_params': '-ip U8 -VPU_MYRIAD_PLATFORM VPU_MYRIAD_2480 -VPU_NUMBER_OF_SHAVES 14 -VPU_NUMBER_OF_CMX_SLICES 14'
}

model_files = {
    'definition': open(xmlfile, 'rb'),
    'weights': open(binfile, 'rb')
}
response = requests.post(url, data=payload, files=model_files, params=params)
blobname = 'deeplabv3p_person.blob.sh14cmx14NCE1'
with open(blob_dir + '/' + blobname, 'wb') as f:
  f.write(response.content)

!wget https://raw.githubusercontent.com/superannotateai/model-deployment-tutorials/main/OAK/deeplabv3p_person.py -O "/content/{OUTPUT_MODEL}/deeplabv3p_person.py"

make_archive(OUTPUT_MODEL, 'zip', OUTPUT_MODEL)
files.download(OUTPUT_MODEL + '.zip')

# 6. Copy and paste the generated model in 'depthai' directory

Please follow these steps on your local machine to get the model running

**1.** Extract the downloaded \$OUTPUT_MODEL.zip archive.

**2.** Move 'deeplabv3p_person.blob.sh14cmx14NCE1' file to \$DEPTHAI_ROOT_DIR/resources/nn/deeplabv3p_person folder.

**3.** Rename the default 'deeplabv3p_person.py' in \$DEPTHAI_ROOT_DIR/depthai_helpers/ folder.

**4.** Move 'deeplabv3p_person.py' file from the extracted archive to \$DEPTHAI_ROOT_DIR/depthai_helpers/ folder.

# 7. Test the custom model

Afterwards you can run the new model with the following command from the $DEPTHAI_ROOT_DIR directory:

```
python3 depthai_demo.py -dd -cnn deeplabv3p_person
```